In [85]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from preprocessing import *

In [86]:
glove_twitter_25 = []
# print("loading pre trained embeddings, this can take some minutes...")
# glove_twitter_25 = KeyedVectors.load_word2vec_format('glove-twitter-25.txt', binary=False)
# print("loading complete.")

In [217]:
w2v_model = Word2Vec.load("w2vPreTrained")

In [256]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t", encoding = "utf-8")
print(dataset.shape)
dataset.dropna(inplace = True)
print(dataset.shape)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True)
print(dataset.shape)
dataset.drop_duplicates(subset = "verified_reviews", inplace = True)
print(dataset.shape)

(3150, 5)
(3150, 5)
(2998, 5)
(2196, 5)


In [257]:
print(dataset["feedback"].value_counts())

1    1990
0     206
Name: feedback, dtype: int64


In [258]:
X = np.array(dataset["verified_reviews"].values).reshape(-1, 1)
y = list(dataset["feedback"].values)

In [259]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state = 0)

X, y = undersampler.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({1: 412, 0: 206})


In [260]:
X_temp = []

for rev in X:
  X_temp.append(rev[0])

In [261]:
new_text, new_sent_tok = tokenize_list_of_text(X_temp, custom_stopwords, False, 2)

total number of types extracted is: 1798


In [262]:
# negative_reviews = list(dataset[dataset["feedback"] == 0]["verified_reviews"].values)
# print(len(negative_reviews))
# negative_text, negative_tok = tokenize_list_of_text(negative_reviews, custom_stopwords, False, pos_list)

In [263]:
# negative_artifical = generate_samples(negative_tok, 200, pre_trained_model = glove_twitter_25)
# new_sent_tok.extend(negative_artifical)
# len(negative_artifical)

In [264]:
cleaned_reviews = frequency_cleaning(new_sent_tok, 2)
# negative_artificial_cleaned = cleaned_reviews[-len(negative_artifical):] # estraggo le recensioni artificiali per poi aggiungerle esclusivamente al train
# del cleaned_reviews[-len(negative_artifical):] # le elimino dalle recensioni pulite

In [265]:
bigrams = Phrases(cleaned_reviews, scoring="npmi", threshold=0.60) #estrae le collocazioni tramite PMI

In [266]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], y, test_size=0.20, random_state=10)
# X_train.extend(negative_artificial_cleaned)
# Y_train.extend([0 for x in range(len(negative_artificial_cleaned))])

In [267]:
print('Train set is %s' % Counter(Y_train))
print('Test set is %s' % Counter(Y_test))

Train set is Counter({1: 329, 0: 165})
Test set is Counter({1: 83, 0: 41})


In [268]:
t = Tokenizer(lower = False)
t.fit_on_texts(X_train)
X_train_encoded = t.texts_to_sequences(X_train)
max_length = len(max(bigrams[cleaned_reviews], key = len))
Xtrain = pad_sequences(X_train_encoded, maxlen=max_length, padding='post')

In [269]:
X_test_encoded = t.texts_to_sequences(X_test)
Xtest = pad_sequences(X_test_encoded, maxlen=max_length, padding='post')

In [270]:
vocab_size = len(t.word_index) + 1

In [271]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, w2v_model.vector_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    not_found = 0
    not_found_list = []
    for word, i in vocab.items():
        try:
            vector = embedding[word]
            weight_matrix[i] = vector
        except KeyError:
            weight_matrix[i] = np.zeros((1, w2v_model.vector_size))
            not_found+=1
            not_found_list.append(word)
            continue

    print(not_found_list)
    print(not_found)
    return weight_matrix

In [272]:
embedding_weights = get_weight_matrix(w2v_model.wv, t.word_index)

['play_music', 'not_a', 'sound_quality', 'differ', 'not_the', 'still_learn', 'listen_music', "'ve", 'light_bulb', 'send_back', "'re", 'even_though', 'uncertain', 'answer_question', 'play_radio', 'alarm_clock', 'unmake', 'look_forward', 'not_very', 'not_that', 'video_chat', "'ll", 'not_i', 'make_life', 'unretentive', 'unaware', 'trailer', 'certified', 'forbid', 'not_this', 'homescreen', 'not_to', 'excitement', 'not_it']
34


In [273]:
from imblearn.under_sampling import RandomUnderSampler, TomekLinks


undersampler_nn = TomekLinks(sampling_strategy="all") # use tomeLinks to remove ambiguous data through nearest neighbours


# Fit and transform the X and y data
# X_train_resampled, y_train_resampled = undersampler_nn.fit_resample(Xtrain, Y_train)
X_train_resampled, y_train_resampled = Xtrain, Y_train


print('Resampled dataset shape %s' % Counter(y_train_resampled))

Resampled dataset shape Counter({1: 329, 0: 165})


In [274]:
from keras.layers import Flatten

In [275]:
from keras.utils import to_categorical
Y_train_hot = to_categorical(y_train_resampled)

In [276]:
print(y_train_resampled[0], Y_train_hot[0])
print(y_train_resampled[1], Y_train_hot[1])

1 [0. 1.]
0 [1. 0.]


In [277]:
Y_test_hot = to_categorical(Y_test)

In [278]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Embedding, Conv1D, MaxPooling1D

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=25)

model = Sequential()
model.add(Embedding(vocab_size, w2v_model.vector_size, input_length=max_length, trainable = True, weights = [embedding_weights]))
model.add(Dropout(0.2))
model.add(Conv1D(filters=16, kernel_size=10, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 100, 100)          61800     
                                                                 
 dropout_40 (Dropout)        (None, 100, 100)          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 91, 16)            16016     
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 45, 16)           0         
 1D)                                                             
                                                                 
 flatten_8 (Flatten)         (None, 720)               0         
                                                                 
 dropout_41 (Dropout)        (None, 720)               0         
                                                      

In [279]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# fit network
# better to fit multiple times
model.fit(X_train_resampled, Y_train_hot, epochs=50, verbose=2, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
13/13 - 3s - loss: 0.6847 - binary_accuracy: 0.6139 - val_loss: 0.6572 - val_binary_accuracy: 0.6515 - 3s/epoch - 246ms/step
Epoch 2/50
13/13 - 0s - loss: 0.5980 - binary_accuracy: 0.7127 - val_loss: 0.6223 - val_binary_accuracy: 0.6717 - 365ms/epoch - 28ms/step
Epoch 3/50
13/13 - 0s - loss: 0.5245 - binary_accuracy: 0.7316 - val_loss: 0.5628 - val_binary_accuracy: 0.6970 - 364ms/epoch - 28ms/step
Epoch 4/50
13/13 - 0s - loss: 0.3889 - binary_accuracy: 0.7937 - val_loss: 0.4873 - val_binary_accuracy: 0.7576 - 366ms/epoch - 28ms/step
Epoch 5/50
13/13 - 0s - loss: 0.3111 - binary_accuracy: 0.8519 - val_loss: 0.5133 - val_binary_accuracy: 0.8081 - 370ms/epoch - 28ms/step
Epoch 6/50
13/13 - 0s - loss: 0.2153 - binary_accuracy: 0.8899 - val_loss: 0.5091 - val_binary_accuracy: 0.8081 - 359ms/epoch - 28ms/step
Epoch 7/50
13/13 - 0s - loss: 0.1936 - binary_accuracy: 0.9101 - val_loss: 0.4789 - val_binary_accuracy: 0.8535 - 360ms/epoch - 28ms/step
Epoch 8/50
13/13 - 0s - loss: 0.1324

In [280]:
predictions = model.predict(Xtest)

4/4 [==============================] - 0s 5ms/step


In [281]:
predictions.shape

(124, 2)

In [282]:
predictions = np.round(predictions)

In [283]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_hot, predictions))

              precision    recall  f1-score   support

           0       0.83      0.73      0.78        41
           1       0.88      0.93      0.90        83

   micro avg       0.86      0.86      0.86       124
   macro avg       0.85      0.83      0.84       124
weighted avg       0.86      0.86      0.86       124
 samples avg       0.86      0.86      0.86       124



In [284]:
x = ["Bad user experience, slow communication and the sound is not strong"]
review, temp = tokenize_list_of_text(x, custom_stopwords, False, 2)
print(temp)
seq_review = t.texts_to_sequences(temp)
padded_review = pad_sequences(seq_review, maxlen=max_length, padding='post')
preds = np.round(model.predict(padded_review))
preds

total number of types extracted is: 7
[['bad', 'user', 'experience', 'slow', 'communication', 'sound', 'weak']]
1/1 [==============================] - 0s 76ms/step


array([[1., 0.]], dtype=float32)

In [285]:
model.predict(padded_review)

1/1 [==============================] - 0s 36ms/step


array([[0.9909038 , 0.00799437]], dtype=float32)

In [78]:
# [1, 0] is equal to negative label